In [90]:
import pandas as pd

## Read and clean CUMTAX csv data

In [91]:
cumatx = pd.read_csv('./input/cumta-route-stop.csv', dtype=str)
cumatx.columns = cumatx.columns.str.strip().str.lower().str.replace('[^a-z0-9]', '_', regex=True)
cumatx = cumatx[['del', 'tummoc_route_id', 'mtc_route_no', 'stop_id', 'sequence', 'name', 'lat', 'lon', 'source', 'destin', 'stage_no', 'stage']]
cumatx = cumatx[cumatx['del'].isna()]

cumatx.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102264 entries, 0 to 104743
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   del              0 non-null       object
 1   tummoc_route_id  102262 non-null  object
 2   mtc_route_no     102262 non-null  object
 3   stop_id          102023 non-null  object
 4   sequence         102262 non-null  object
 5   name             102262 non-null  object
 6   lat              102262 non-null  object
 7   lon              102262 non-null  object
 8   source           102262 non-null  object
 9   destin           102262 non-null  object
 10  stage_no         102250 non-null  object
 11  stage            41043 non-null   object
dtypes: object(12)
memory usage: 10.1+ MB


In [92]:
route_counts = cumatx.groupby('tummoc_route_id').size().reset_index(name='count').sort_values('count', ascending=True)
route_counts


,tummoc_route_id,count
1555,316,2
2805,4438,2
2897,4525,2
731,2297,2
3008,4631,2
...,...,...
656,2214,69
1473,3080,71
3161,4830,72
3582,76,76


In [93]:
route_counts[route_counts['count'] > 2]


,tummoc_route_id,count
513,2068,3
2178,3796,3
254,1741,3
2068,3683,3
3330,5033,3
...,...,...
656,2214,69
1473,3080,71
3161,4830,72
3582,76,76


## Read and clean Route Stop Mapping from GTFS Dashboard

In [94]:
gtfs_dashboard_df = pd.read_csv('./input/new-route-stop.csv', dtype=str)
gtfs_dashboard_df.columns = gtfs_dashboard_df.columns.str.strip().str.lower().str.replace('[^a-z0-9]', '_', regex=True)
gtfs_dashboard_df = gtfs_dashboard_df[gtfs_dashboard_df['stop_type'] != 'ROUTE CORRECTION']
gtfs_dashboard_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86574 entries, 0 to 86880
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   route_id      86574 non-null  object
 1   route_number  86574 non-null  object
 2   stop_id       86574 non-null  object
 3   stop_name     86574 non-null  object
 4   latitude      86574 non-null  object
 5   longitude     86574 non-null  object
 6   sequence      86574 non-null  object
 7   stop_type     86574 non-null  object
 8   stage_no      86574 non-null  object
 9   stage_name    84671 non-null  object
 10  provider_id   86574 non-null  object
dtypes: object(11)
memory usage: 7.9+ MB


## Check for missing routes in GTFS Dashboard

In [95]:
missing_routes = cumatx[~cumatx['tummoc_route_id'].isin(
    gtfs_dashboard_df['route_id'])]['tummoc_route_id'].unique()
missing_routes_df = pd.DataFrame(missing_routes, columns=['tummoc_route_id'])
missing_routes_df

,tummoc_route_id
0,3
1,336
2,335
3,1950
4,1951
...,...
525,1863
526,1892
527,1891
528,1866


## Add missing route info to GTFS Dashboard df

In [96]:
out_df = gtfs_dashboard_df.copy()

# Get the rows from cumatx that correspond to missing routes
missing_route_data = cumatx[cumatx['tummoc_route_id'].isin(missing_routes)].copy()

# Create stop_type column based on stage values using loc to avoid SettingWithCopyWarning
missing_route_data.loc[:, 'stop_type'] = missing_route_data['stage'].apply(
    lambda x: 'NEW STOP' if pd.notna(x) else 'INTERMEDIATE'
)

# Map the column names between cumatx and out_df
column_mapping = {
    'tummoc_route_id': 'route_id',
    'mtc_route_no': 'route_number',
    'stop_id': 'stop_id',
    'name': 'stop_name',  # Changed from 'stop_name': 'name'
    'lat': 'latitude',
    'lon': 'longitude',
    'sequence': 'sequence',
    'stage_id_no': 'stop_type',
    'stage': 'stage_name',
    'stage_id_no': 'stage_no'
}

# Rename columns in missing_route_data to match out_df
missing_route_data = missing_route_data.rename(columns=column_mapping)

missing_route_data.to_csv('./output/missing_route_data.csv', index=False)

In [97]:
# Create empty columns for any missing columns from out_df
for col in out_df.columns:
    if col not in missing_route_data.columns:
        missing_route_data[col] = None

# # Concatenate the missing route data with out_df
out_df = pd.concat([out_df, missing_route_data], ignore_index=True)

In [98]:
# Forward fill stage values in out_df
out_df['stage_name'] = out_df['stage_name'].ffill()

# Forward fill stage_no values based on stage values
current_stage = None
current_stage_no = None

for idx, row in out_df.iterrows():
    if pd.notna(row['stage_name']) and row['stage_name'] != current_stage:
        current_stage = row['stage_name']
        current_stage_no = row['stage_no']
    elif current_stage_no is not None:
        out_df.at[idx, 'stage_no'] = current_stage_no

## Update the route_id, route_number mapping

In [99]:
route_number_mapping = pd.read_csv('./input/RoutePointMaster.csv', dtype=str)
route_number_mapping.columns = route_number_mapping.columns.str.strip().str.lower().str.replace('[^a-z0-9]', '_', regex=True)
# Trim spaces from all columns
for col in route_number_mapping.columns:
    if route_number_mapping[col].dtype == 'object':
        route_number_mapping[col] = route_number_mapping[col].str.strip()

route_number_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52890 entries, 0 to 52889
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              52890 non-null  object
 1   route_number     52890 non-null  object
 2   source           52890 non-null  object
 3   destination      52890 non-null  object
 4   route_name       52890 non-null  object
 5   route_direction  52890 non-null  object
 6   bus_stop_name    52890 non-null  object
 7   route_id         52890 non-null  object
 8   bus_stop_id      52890 non-null  object
 9   route_order      52890 non-null  object
 10  stage_no         52890 non-null  object
dtypes: object(11)
memory usage: 4.4+ MB


In [100]:
route_counts = route_number_mapping.groupby('_id').size().reset_index(name='count').sort_values('count', ascending=True)
route_counts

,_id,count
5077,78,1
5242,930,1
5241,93,1
2641,353,1
1438,244,1
...,...,...
2436,3345,30
2435,3344,30
5213,904,30
5110,809,30


In [101]:
# Create a mapping dictionary from _id to route_number
route_id_to_number = dict(zip(route_number_mapping['_id'], route_number_mapping['route_number']))

# Update route_number in out_df using the mapping
out_df['route_number'] = out_df['route_id'].map(route_id_to_number)


## Final output

In [102]:
out_df.to_csv('./output/updated_route_stop.csv', index=False)